In [ ]:
import org.apache.spark

In [ ]:
val path_to_bigDatasets = "../../../../datasets/big/"
val path_to_datasets = "../../../../datasets/"

val path_Fullml_posts = path_to_bigDatasets + "the-reddit-covid-dataset-posts.csv"
val path_Fullml_comments = path_to_bigDatasets + "the-reddit-covid-dataset-comments.csv"

val path_sample_posts = path_to_datasets + "postsSample.csv"
val path_sample_comments = path_to_datasets + "commentsSample.csv"

In [ ]:
import java.util.Calendar
import java.text.SimpleDateFormat
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner

object CovidConversationsParser {

    val commaSplit = ","
    val quotes = "\""

    /** Convert from timestamp (String) to Month + day (String) */
    def monthDayFromTimestamp(timestamp: String): String = {
        val cal = Calendar.getInstance()
        cal.setTimeInMillis(timestamp.trim.toLong * 1000L)
        val format = new SimpleDateFormat("MMMM")
        format.format(cal.getTime()) + ": " + cal.get(Calendar.DAY_OF_MONTH)
    }

    /** Function to parse reddit posts
    *
    *  @param line line that has to be parsed
    *  @return tuple containing id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score. none in case of input errors
    */
    def parseRedditPost(line: String): Option[(String, String, String, Boolean, String, String, String, String, String, String, Int)] = {
        try {
            val input = line.split(commaSplit)
            if(input.size != 12) {
                return Some(("", "", "", false, "", "", "", "", "", "", -1))
            }
            var url = "None"
            if(input(8).trim.nonEmpty) {
                url = input(8).trim
            }
            var selftext = "None"
            if(input(9).trim.nonEmpty) {
                selftext = input(9).trim
            }
            val number = input(11).trim.replaceAll(quotes, "")
            Some((input(1).trim, input(2).trim, input(3).trim, input(4).trim.toBoolean, monthDayFromTimestamp(input(5)), input(6).trim, input(7).trim, url, selftext, input(10).trim , number.toInt))
        } catch {
            case _: Exception => None
        }
    }

    /** Function to parse reddit comments
    *
    *  @param line line that has to be parsed
    *  @return tuple containing id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score. none in case of input errors
    */
    def parseRedditComment(line: String): Option[(String, String, String, Boolean, String, String, String, Double, Int)] = {
        try {
            val input = line.split(commaSplit)
            if(input.size != 10) {
                return Some(("", "", "", false, "", "", "", -0.1, -1))
            }
            val number = input(9).trim.replaceAll(quotes, "")
            Some((input(1).trim, input(2).trim, input(3).trim, input(4).trim.toBoolean, monthDayFromTimestamp(input(5)), input(6).trim, input(7).trim, input(8).trim.toDouble, number.toInt))
        } catch {
            case _: Exception => None
        }
    }
}

In [ ]:
//per recuperare dei samples dei files
//sc.textFile(path_Fullml_posts).sample(false, 0.05).coalesce(1).toDF().write.format("csv").mode(SaveMode.Overwrite).save("../../../../datasets/sample")
sc.textFile(path_Fullml_comments).sample(false, 0.02).coalesce(1).toDF().write.format("csv").mode(SaveMode.Overwrite).save("../../../../datasets/sample")

In [ ]:
/*getting RDDs of Comments and Posts*/
val rddPosts = sc.textFile(path_sample_posts).flatMap(CovidConversationsParser.parseRedditPost).filter(x => x != ("","","",false, "","","","","","",-1))
val rddComments = sc.textFile(path_sample_comments).flatMap(CovidConversationsParser.parseRedditComment)
.filter(x => x != ("", "", "", false, "", "", "", -0.1, -1))
rddComments.first()

In [ ]:
//PART 1: Aggregate on temporal dimension and obtain percentage of posts classified as NSFW
//Posts are :(id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score)
/* solution is better using  reduceByKey becasue:
    -Avoid collecting all values into memory before processing
    -Perform aggregation as data is being processed
    -Reduce network shuffle by combining data locally first
*/
val percentageNSFWPosts = rddPosts.map(x => (x._5, (1, if (x._4) 1 else 0))) // (created_utc, (posts count, nsfw flag))
.reduceByKey ({ case ((total1, nsfw1), (total2, nsfw2)) =>
(total1 + total2, nsfw1 + nsfw2)
})
.mapValues ({ case (total, nsfw) =>
    //Calculate the percentage
    val percentage = (nsfw.toDouble * 100) / total
    "Percentage NSFW Posts: " + percentage + "%"
})

//.coalesce(1) //riduzione a singola partizione (per analisi, con singola partizione abbiamo singolo file da analizzare)
//.collect()

In [ ]:
//PART 2: Aggregate on temporal dimension and obtain average sentiment in comments and percentage of comments classified as NSFW
//Comments are: (id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score)

val avgSentimentWithNSFWComment = rddComments.map(x => (x._5, (if (x._4) 1 else 0, 1, x._8))) //(created_utc, nsfw flag, number of posts, sentiment)
.reduceByKey((a, b) => (
    a._1 + b._1,  // Sum NSFW counters
    a._2 + b._2,  // Sum total post counters
    a._3 + b._3   // Sum sentiment values
))
.mapValues(reduced => {
    val (nsfwCount, totalCount, totalSentiment) = reduced
    val nsfwPercentage = (nsfwCount * 100.0) / totalCount
    val avgSentiment = totalSentiment / totalCount
    ("Percentage NSFW Comments: "+ nsfwPercentage + "%", "Average Sentiment: " + avgSentiment)
})
//.coalesce(1) //riduzione a singola partizione (per analisi, con singola partizione abbiamo singolo file da analizzare)
//.collect()

In [ ]:
val path_output = "../../../../output/covidPostCommentResults"
sc.getPersistentRDDs.foreach(_._2.unpersist())

In [ ]:
// PART 3: Join on Temporal Dimension and then write the output

val finalResult = percentageNSFWPosts.join(avgSentimentWithNSFWComment) 
//with the join i get an Array[(String,(String,(String, String)))], in order to properly write it, i map it first then change it
.map(x => (x._1, x._2._1, x._2._2._1, x._2._2._2))
.coalesce(1) //riduzione a singola partizione (per analisi, con singola partizione abbiamo singolo file da analizzare)
.toDF().write.format("csv").mode(SaveMode.Overwrite).save(path_output)